Can we quantify the carbon cost savings by having Access Virtual this year?

Let's start with some information.  

> According to the [American Council for an Energy-Efficient Economy](https://www.emergeinteractive.com/wp-content/uploads/2020/02/aceee-cached-research.pdf) it takes 5.12 kWh of electricity per gigabyte of transferred data. And according to the [Department of Energy](https://www.emergeinteractive.com/wp-content/uploads/2020/02/DOE-CO2-Emissions-Generation-Electric-Power-2000.pdf) the average US power plant expends 600 grams of carbon dioxide for every kWh generated. That means that transferring 1GB of data produces 3kg of CO2. [https://www.emergeinteractive.com/insights/detail/does-irresponsible-web-development-contribute-to-global-warming/]


Originally I thought that this is our average
> A one-hour high-definition video call consumes about 540 MB per person. [https://gerrymcgovern.com/the-hidden-pollution-cost-of-online-meetings/]

But some observation shows that it's more like for me.  
![](images/access_goat_live_stream.PNG)
I'm not sure if there would be variation in that per participant. Obviously the speakers would also be sending data. For now assuming that that value is negligable. This was on the higher end of what I observed.  I think 1000 kb/s is a more acurate value.

Access Conference runs 28.5 hours
- Monday: 8:30 to 3:30 (7 hours)
- Tuesday: 8:30 to 1:30 (5 hours)
- Wednesday: Hackfest and Workshops 8:30 to 3:30 (7 hours)
- Thursday: 8:30 to 1:30 (5 hours)
- Friday: 8:30 to 1 (4.5 hours)

The conference is recorded and live streamed either way so assume that these costs are the same. We have to assume that the same people would participate but we know this isn't true.  The conference being virtual (and free) made it possible for attendees without a PD or travel budget able to attend.

> Since each tree will remove 7 tons of Carbon Dioxide [https://ythakker.medium.com/how-many-new-trees-would-it-take-to-offset-our-carbon-emissions-13c78ccb09ab]

In [1]:
require 'csv'

input = CSV.read('data/2020-registrations_locations.csv', headers: :first_row).map(&:to_h)

total_registrations = 0
input.each do |row|
    total_registrations += row['count'].to_i
end

puts "total registrations: #{total_registrations}"

total registrations: 464


~Let's say that there is 60% attendance at any given time.~  By observation there were 25% attendance on Thursday.  Even if we say this is 30%

28.5 hours \* 464 people \* 30% attendance \* 1 MB/(person seconds) \* 0.001 GB/MB \* 60 seconds/minute \* 60 minutes/hour \* 3 kg CO2e/GB \* 0.001 tonne/kg

In [2]:
"#{28.5*464*0.3*1*0.001*60*60*3*0.001} tonne CO2e for the 2020 Access Conference"

"42.845760000000006 tonne CO2e for the 2020 Access Conference"

In [3]:
require 'geocoder'

Geocoder.configure(:units => :km)

vancouver = Geocoder.search('Vancouver, BC, Canada')
confcoords = [vancouver.first.data['lat'],vancouver.first.data['lon']]

input = CSV.read('data/2020-registrations_locations.csv', headers: :first_row).map(&:to_h)

total_distance = 0
distance_less_than_1000 = 0
people_travelleing_less_than_1000 = 0
distance_greater_than_1000 = 0
people_travelleing_greater_than_1000 = 0

input.each do |row|
    geocoded = Geocoder.search(row['location'])
    lat = geocoded.first.data['lat']
    lon = geocoded.first.data['lon']    
    distance = Geocoder::Calculations.distance_between(confcoords,[lat, lon]).round
    if distance < 1000
        distance_less_than_1000 += distance*row['count'].to_i
        people_travelleing_less_than_1000 +=row['count'].to_i
    else
        distance_greater_than_1000 += distance*row['count'].to_i
        people_travelleing_greater_than_1000 +=row['count'].to_i
    end
    total_distance += distance*row['count'].to_i
end

puts "total distance travelled: #{total_distance} km"
puts "total distance flown: #{distance_greater_than_1000} km (average: #{distance_greater_than_1000/people_travelleing_greater_than_1000} km) by #{people_travelleing_greater_than_1000} people"
puts "total distance driven: #{distance_less_than_1000} km (average: #{distance_less_than_1000/people_travelleing_less_than_1000} km) by #{people_travelleing_less_than_1000} people"

total distance travelled: 1034546 km
total distance flown: 988817 km (average: 3469 km) by 285 people
total distance driven: 45729 km (average: 255 km) by 179 people


Suppose everyone that was closer than 1000 km drove and greater than 1000 km flew. Assume that no one carpooled.  Also assume that everyone flew economy.

> The average new car emits 120.1g/km of CO2 [https://www.lightfoot.co.uk/news/2017/10/04/how-much-co2-does-a-car-emit-per-year/]


Trying to find an estimate for the flights so far the best I've found is using a flight with the average distance.  Turns out Vancouver to New Orleans is about that far but there aren't any direct flights.  The distance is somewhere between Toronto and Ottawa. Let's use Ottawa's values.

|Dep Airport|Arr Airport|Distance (KM)|Aircraft|Aircraft Fuel Burn/leg (KG)|Passenger CO2/pax/leg (KG)|
|--|--|--|--|--|--|
|YVR|YOW|3550.0|319, 320, 321, 736, 73H, 73W|13805.5|275.3|
|YOW|YVR|3550.0|319, 320, 321, 736, 73H, 73W|13786.8|275.6|

[https://www.icao.int/environmental-protection/CarbonOffset/Pages/default.aspx]

((120.1 g CO2e/km \* 45729 km \* 0.001 kg/g) + (275.3 kg CO2e + 275.6 kg CO2e)/person \* 285 people))\* 0.001 tonne/kg

In [4]:
"#{((120.1 * 45729  * 0.001) + ((275.3+275.6) * 285))* 0.001} tonne CO2e if Access had been in Vancouver"

"162.49855290000005 tonne CO2e if Access had been in Vancouver"

- Virtual Access (🌲🌲🌲🌲🌲🌲): 42.8 tonnes CO2e / 7 tonnes CO2e per tree ~6
- Physical Access (🌲🌲🌲🌲🌲🌲🌲🌲🌲🌲🌲🌲🌲🌲🌲🌲🌲🌲🌲🌲🌲🌲🌲): 162.5 tonnes CO2e / 7 tonnes CO2e per tree ~23